## Open file and select text column

In [ ]:
!pip install nltk
!pip install textblob

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer, StopWordsRemover
import pandas as p
import nltk
from textblob import TextBlob

# Initialize the Spark session, no configurations?
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Twitter") \
    .getOrCreate()

# Setup Hadoop file system configuration
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopF5")

# Google storage filepath
gsc_file_path = 'gs://dejadsdejads_group10assignment2/vaccination_tweets.csv'
tweet_table = (spark.read
      .option("multiline", "true")
      .option("quote", '"')
      .option("header", "true")
      .option("escape", "\\")
      .option("escape", '"')
      .csv(gsc_file_path)
)

# Print schema and number of tweets in the dataset
print("Table schema: {}\n".format(tweet_table.schema))
print("Number of tweets: {}\n".format(tweet_table.count()))

# Select tweet text and peak into the text
tweet_text_table = tweet_table.select(['text'])
tweet_text_table.show(10,truncate=False)

ModuleNotFoundError: No module named 'nltk'

### Cleaning

In [63]:
# Removing empty tweets
tweet_text_table = tweet_text_table.na.drop()
print("Number of tweets after removing null: {}\n".format(tweet_text_table.count()))

# Remove twitter handlers, hashtags, URLS, special characters, single characters and double spaces respectively
df_clean = tweet_text_table.select('text', ((regexp_replace('text', '@[^\s]+', "")).alias('text2')))
df_clean = df_clean.select('text2', ((regexp_replace('text2', r'\B#\S+', "")).alias('text3')))
df_clean = df_clean.select('text3', ((regexp_replace('text3', r"http\S+", "")).alias('text4')))
df_clean = df_clean.select('text4', ((regexp_replace('text4', r'[^\w+]', " ")).alias('text5'))) 
df_clean = df_clean.select('text5', ((regexp_replace('text5', r'\s+[a-zA-Z]\s+', " ")).alias('text6')))
df_clean = df_clean.select('text6', ((regexp_replace('text6', r'\s+', " ")).alias('text'))).select(['text'])

# Create index
df_clean = df_clean.select("*") \
                    .withColumn("id", monotonically_increasing_id()) \
                    .select("id", "text", "date")

# Peak
df_clean.show(10,truncate=False)

Number of tweets after removing null: 7632

+---+------------------------------------------------------------------------------------------------------------------+
|id |text                                                                                                              |
+---+------------------------------------------------------------------------------------------------------------------+
|0  |Same folks said daikon paste could treat cytokine storm                                                           |
|1  |While the world has been on the wrong side of history this year hopefully the biggest vaccination effort we ve ev |
|2  | Russian vaccine is created to last 2 4 years                                                                     |
|3  |Facts are immutable Senator even when you re not ethically sturdy enough to acknowledge them 1 You were born      |
|4  |Explain to me again why we need vaccine                                                                 

## Sentiment analysis

In [61]:
# There was an issue with the texblob module such that it did not allow for application of an udf to an entire column. 
# To fix this, we converted the pyspark df to a pandas df and simply appended a list of sentiments to the df.

pandas_df = df_clean.toPandas()
sentiment_list = list()

for index, row in pandas_df.iterrows():
    sentiment = row['text']
    sentiment_list.append(TextBlob(sentiment).sentiment[0])

sentiments_df = spark.createDataFrame(sentiment_list, FloatType()) \
                    .select("*") \
                    .withColumn("id", monotonically_increasing_id()) 

# Join dataframes
tweet_sentiments_df = df_clean.join(sentiments_df, "id").withColumnRenamed("value", "sentiment")

# Peak
tweet_sentiments_df.show(10, truncate=False)

+---+------------------------------------------------------------------------------------------------------------------+------------------+
|id |text                                                                                                              |value             |
+---+------------------------------------------------------------------------------------------------------------------+------------------+
|0  |Same folks said daikon paste could treat cytokine storm                                                           |0.0               |
|1  |While the world has been on the wrong side of history this year hopefully the biggest vaccination effort we ve ev |-0.5              |
|2  | Russian vaccine is created to last 2 4 years                                                                     |0.0               |
|3  |Facts are immutable Senator even when you re not ethically sturdy enough to acknowledge them 1 You were born      |-0.05             |
|4  |Explain to me a

The next step is to save the sentiment analysis df to BigQuery.

In [ ]:
# Use the Cloud Storage bucket for temporary BigQuery export
bucket = "dejadsdejads_group10assignment2"
spark.conf.set('temporaryGcsBucket', bucket)

# Saving the data to BigQuery
tweet_sentiments_df.write.format('bigquery') \
  .option('table', 'de2021labs.twitter_data.twitter_sentiments') \
  .mode("overwrite") \
  .save()

# Word Frequency
Besides sentiment analysis, a word count is also performed to understand the data better.

In [ ]:
df_clean.select("text").show(10)

In [ ]:
import pyspark.sql.functions as f
from pyspark.ml.feature import Tokenizer, StopWordsRemover

# Tokenize
tokenizer = Tokenizer(inputCol='text', outputCol='body_tokenized')
twitter_data_text_tokenized = tokenizer.transform(df_clean).select('id', 'body_tokenized')

# Remove unnecessary words
unncessary_words_list = ["like", "know", "get", 'one', 'think', 'cause', 'say', 'even', "don't", 'got', 'also', 'good', 'said',
                        'make', 'it.', 'first', 'many', 'still', 'actually', "don't", 'want', 'read', 'print', 'vaccine',
                        'vaccines', 'vaccinated', 'vaccination', 'may', 'saying', 'point', 'virus', 'never', 'much', 'see',
                        '1', 'way', 'wrong', 'really', 'used', 'well', 'getting', 'take', 'every', 'go', '>'] 
unncessary_words_list.extend(StopWordsRemover().getStopWords())
remover = StopWordsRemover(inputCol='body_tokenized', outputCol='body_clean', stopWords=unncessary_words_list)
twitter_data_text_no_stopwords = remover.transform(twitter_data_text_tokenized).select(['id', 'body_clean'])

# Return to regular strings
twitter_data_text_no_stopwords = twitter_data_text_no_stopwords.withColumn("body_clean", 
                                                                         concat_ws(",", "body_clean"))

# Count the words
count_twitter = twitter_data_text_no_stopwords.withColumn('body_clean', f.explode(f.split(f.col('body_clean'), ',')))\
    .groupBy('body_clean')\
    .count()\
    .sort('count', ascending=False)\
    .filter(f.col('body_clean') != "")\
    .withColumnRenamed("body_clean", "word")

count_twitter.show()

In [ ]:
# Save the word count
count_twitter.write.format('bigquery') \
  .option('table', 'de2021labs.twitter_data.twitter_wordcount') \
  .mode("overwrite") \
  .save()

At last, the spark context should be stopped.

In [ ]:
spark.stop()